In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.initializers import GlorotNormal

In [3]:
train_dir = '/kaggle/input/fruits-dataset-images/images'

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 359 images belonging to 9 classes.


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(9, activation='softmax')
])

**I tried using He and Glorot Initialization techniques for weights, but I was getting accuracies around 0.6 to 0.7. However, with random initialization, my highest accuracy was 0.84**

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
num_epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    verbose=1
)

Epoch 1/10
12/12 [==============================] - 20s 2s/step - loss: 1.1279 - accuracy: 0.6156
Epoch 2/10
12/12 [==============================] - 20s 2s/step - loss: 0.9539 - accuracy: 0.6212
Epoch 3/10
12/12 [==============================] - 20s 2s/step - loss: 0.8811 - accuracy: 0.6964
Epoch 4/10
12/12 [==============================] - 20s 2s/step - loss: 0.8207 - accuracy: 0.7075
Epoch 5/10
12/12 [==============================] - 20s 2s/step - loss: 0.8088 - accuracy: 0.6936
Epoch 6/10
12/12 [==============================] - 20s 2s/step - loss: 0.8122 - accuracy: 0.7075
Epoch 7/10
12/12 [==============================] - 19s 2s/step - loss: 0.8640 - accuracy: 0.6741
Epoch 8/10
12/12 [==============================] - 20s 2s/step - loss: 0.6402 - accuracy: 0.7744
Epoch 9/10
12/12 [==============================] - 20s 2s/step - loss: 0.6586 - accuracy: 0.7577
Epoch 10/10
12/12 [==============================] - 20s 2s/step - loss: 0.5684 - accuracy: 0.8022


In [17]:
train_loss, train_accuracy = model.evaluate(train_generator, steps=len(train_generator))

print(f"Training Loss: {train_loss:.4f}")
print(f"Training Accuracy: {train_accuracy:.4f}")

12/12 [==============================] - 15s 1s/step - loss: 0.5345 - accuracy: 0.7967
Training Loss: 0.5345
Training Accuracy: 0.7967


In [10]:
model.save('/kaggle/working/myFruitclassifier001')  

In [11]:
# from IPython.display import FileLinks
# #
# # Create a download link for the model file
# FileLinks('/kaggle/working/myFruitclassifier001') 

In [12]:
# import zipfile

# # Define the path to your SavedModel directory
# saved_model_dir = '/kaggle/working/myFruitclassifier001'

# # Define the name of the ZIP archive
# zip_filename = '/kaggle/working/myFruitclassifier001.zip'

# # Create a ZIP archive of the SavedModel directory
# with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
#     for root, dirs, files in os.walk(saved_model_dir):
#         for file in files:
#             zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), saved_model_dir))

# zip_filename  


In [13]:
# from IPython.display import FileLink

# FileLink(zip_filename)

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_fruit_class(image_path, model_path='/kaggle/working/myFruitclassifier001'):
    # Load the trained model
    model = tf.keras.models.load_model(model_path)

    # Load and preprocess the new image
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array /= 255.0  # Normalize pixel values

    # Expand dimensions to match the model's input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions
    predictions = model.predict(img_array)

    # Get the predicted class index
    predicted_class_index = np.argmax(predictions[0])

    class_labels = ['apple fruit', 'banana fruit', 'cherry fruit', 'chickoo fruit', 'grapes fruit', 'kiwi fruit', 'mango fruit', 'orange fruit', 'strawberry fruit']

    # Get the predicted class label
    predicted_class_label = class_labels[predicted_class_index]

    return predicted_class_label

# Example usage:
image_path = '/kaggle/input/strawberry/three-strawberries-strawberry-leaf-white-background-114284301.jpg.webp'
predicted_class = predict_fruit_class(image_path)
print(f"Predicted Class: {predicted_class}")


1/1 [==============================] - 0s 149ms/step
Predicted Class: strawberry fruit


In [15]:
predict_fruit_class('/kaggle/input/strawberry/three-strawberries-strawberry-leaf-white-background-114284301.jpg.webp')

1/1 [==============================] - 0s 105ms/step


'strawberry fruit'